# 1. softmax回归的简洁实现

通过深度学习框架的高级API能够使实现softmax回归变得更容易

In [1]:
import torch 
from torch import nn
import d2l_torch as d2l

batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)

In [20]:
# 参见额外扩展中->代码中的flatten验证部分说明
net=nn.Sequential(nn.Flatten(),nn.Linear(784,10))

def init_weights(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
        # 使用正态分布对某个层进行初始化，mean默认为0，std默认为1
        print(m.weight[:5,:5],m.weight.shape)
net.apply(init_weights)
# 对调用者的每个子模块应用函数，最典型的使用就是模型初始化的时候

tensor([[ 0.0089,  0.0113, -0.0199,  0.0070, -0.0123],
        [-0.0041,  0.0079,  0.0057,  0.0080,  0.0029],
        [ 0.0103,  0.0111,  0.0264, -0.0135,  0.0078],
        [ 0.0047,  0.0201, -0.0006, -0.0007,  0.0184],
        [ 0.0172,  0.0051,  0.0040, -0.0029,  0.0116]],
       grad_fn=<SliceBackward>) torch.Size([10, 784])


Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
)

可以看到，`Linear`网络的输入输出是784\*10，刚好和`Linear`的weight形状10\*784是对称的。

In [ ]:
# 参考 额外扩展 -> pytorch中的CrossEntropyLoss
# 在交叉熵损失函数中传递未归一化的预测，并同时计算softmax及其对数。
loss=nn.CrossEntropyLoss()

**总结**

简洁实现和从0实现之间的对应关系

|简洁实现|从0实现|作用|
|---|---|---|
|nn.Flatten() |reshape |将输入数据变成二维的|
|init_weights()| 对W和b赋值|初始化|
|CrossEntropyLoss|-torch.log(softmax())|加入-log的softmax作为损失函数|

# 额外扩展

## 代码中的flatten()验证

In [9]:
for X,y in train_iter:
    print(X.shape)
    break

torch.Size([256, 1, 28, 28])


softmax回归的 输出层 是一个全连接层。

+ 由于pytorch不会隐式的帮助我们调整输入的形状
+ 因此，定义展平层(flatten)在线性层前调整网络输入的形状
+ flatten的作用是把输入的任意维度的tensor变成一个2d的tensor，其中输入tensor的第0维保留，剩下的维度全部展成一个向量。
    + 函数说明中：Args:
        + start_dim: first dim to flatten (default = 1).
        + end_dim: last dim to flatten (default = -1).
    + 这个flatten就对应softmax从头实现里的reshape函数
+ train_iter中每个batch的size如上，其实是256个 通道数为1，大小为28\*28的图像
+ 但是由于Linear网络层接收的输入输出只能是一维的，所以使用Flatten压扁，变成256\*784，如下

In [12]:
for X,y in train_iter:
    m = nn.Sequential(nn.Flatten())
    output = m(X)
    print(output.size())
    break

torch.Size([256, 784])


## flatten()和conv2d()函数

关于flatten()，可以看下面这个例子
+ 输入是32个1通道 5\*5的图像
+ 使用的2d卷积，
    + `in_channel`=1,
    + `out_channels`=32
    + `kernel_size`=5\*5
    + `stride` = 1,
    + `padding`= 1,

In [2]:
input = torch.randn(32, 1, 5, 5)
m = nn.Sequential(
    nn.Conv2d(1, 32, 5, 1, 1),
    nn.Flatten()
)
output = m(input)
output.size()

torch.Size([32, 288])

288=32\*3\*3

In [6]:
input = torch.randn(32, 1, 5, 5)
m = nn.Sequential(nn.Conv2d(1, 32, 5, 1, 1))
output = m(input)
output.size()

torch.Size([32, 32, 3, 3])

## pytorch中的CrossEntropyLoss

从0实现中，是cross_entropy调用了softmax。而pytorch对于交叉熵损失函数的实现是：
[torch.nn.CrossEntropyLoss.html](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)